In [ ]:
from openff.toolkit.topology import Molecule, Topology

from openff.system.stubs import ForceField
from openff.system.utils import get_test_file_path

In [ ]:
# Create an OpenFF Topology consisting of two ethanol molecules
top = Topology.from_molecules(2 * [Molecule.from_smiles("CCO")])

In [ ]:
# Load in two versions of OpenFF 1.x.x ("Parsley")
parsley_100 = ForceField("openff-1.0.0.offxml")
parsley_130 = ForceField("openff-1.3.0.offxml")

In [ ]:
# Use a monkey-patched function to parametrize the topology against each force field
sys_100 = parsley_100.create_openff_system(top)
sys_130 = parsley_130.create_openff_system(top)

In [ ]:
# Look into each system's angle values ...
sys_100.handlers["Angles"].potentials

In [ ]:
# ... and notice that they're (slightly) different values
sys_130.handlers["Angles"].potentials

In [ ]:
# This can be verified by directly comparing the objects
assert not sys_100.handlers["Angles"] == sys_130.handlers["Angles"]

In [ ]:
# (This first step not strictly necessary, as the typing did not change between versions of this force field line)
sys_100.handlers["Angles"].store_matches(parsley_100["Angles"], topology=top)

# But we can replace the `Potential` objects with openff-1.3.0 values by feeding it in
# the "Bonds" section of that force field, which is represented in memory by a `BondHandler` object
sys_100.handlers["Angles"].store_potentials(parsley_130["Angles"])

In [ ]:
assert sys_100.handlers["Angles"] == sys_130.handlers["Angles"]

In [ ]:
# Or, more verbosely, we can again inspect the objects themselves ...
sys_100.handlers["Angles"].potentials

In [ ]:
# ... and see that they are now both using the `openff-1.3.0` values
sys_130.handlers["Angles"].potentials

In [ ]:
# But we didn't change any other values, i.e. bonds
assert not sys_100.handlers["Bonds"] == sys_130.handlers["Bonds"]

In [ ]:
# TODO: Add a trip to disk